In [0]:
from google.colab import files
uploaded = files.upload()

Saving seg.zip to seg.zip


In [0]:
import zipfile
import io
zf = zipfile.ZipFile(io.BytesIO(uploaded["seg.zip"]), "r")
zf.extractall()

In [0]:
!ls

datalab  melanoma_data.zip  seg_data.pkl  seg.zip


In [0]:
from keras.optimizers import Adam
import keras
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from keras.models import Sequential, Model
from keras.layers import Conv2DTranspose, concatenate, Input,  Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import numpy as np
import random
from keras import backend as K
smooth = 1
import cv2
import os
import pickle

Using TensorFlow backend.


In [0]:
d = pickle.load(open( 'seg_data.pkl', 'rb'))
data = d['data']
labels = d['labels']

In [0]:
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels, dtype="float") / 255.0

(x_train, x_test, y_train, y_test) = train_test_split(data,labels, test_size=0.20, random_state=42)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

1600 train samples
400 test samples


In [0]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)


batch_size = 32
num_classes = 2
epochs = 50

img_rows, img_cols = 64, 64

input_shape = (img_rows, img_cols,3)

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

inputs = Input((img_rows, img_cols, 3))

conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

conv10 = Conv2D(3, (1, 1), activation='sigmoid')(conv9)

model = Model(inputs=[inputs], outputs=[conv10])

model.compile(optimizer=Adam(lr=1e-5), loss=dice_coef_loss, metrics=[dice_coef])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)

print('Sorensen–Dice_coefficient:', score[1])

x_train shape: (1600, 64, 64, 3)
1600 train samples
400 test samples
Train on 1600 samples, validate on 400 samples
Epoch 1/50
1600/1600 [==============================] - 13s 8ms/step - loss: -0.2771 - dice_coef: 0.2771 - val_loss: -0.2881 - val_dice_coef: 0.2881
Epoch 2/50
1600/1600 [==============================] - 9s 6ms/step - loss: -0.2802 - dice_coef: 0.2802 - val_loss: -0.2911 - val_dice_coef: 0.2911
Epoch 3/50
1600/1600 [==============================] - 9s 6ms/step - loss: -0.2847 - dice_coef: 0.2847 - val_loss: -0.3009 - val_dice_coef: 0.3009
Epoch 4/50
1600/1600 [==============================] - 9s 6ms/step - loss: -0.3274 - dice_coef: 0.3274 - val_loss: -0.3596 - val_dice_coef: 0.3596
Epoch 5/50
 448/1600 [=======>......................] - ETA: 6s - loss: -0.3524 - dice_coef: 0.3524

1600/1600 [==============================] - 9s 6ms/step - loss: -0.3704 - dice_coef: 0.3704 - val_loss: -0.4395 - val_dice_coef: 0.4395
Epoch 6/50
1600/1600 [==============================] - 9s 6ms/step - loss: -0.5459 - dice_coef: 0.5459 - val_loss: -0.5764 - val_dice_coef: 0.5764
Epoch 7/50
1600/1600 [==============================] - 9s 6ms/step - loss: -0.5825 - dice_coef: 0.5825 - val_loss: -0.5807 - val_dice_coef: 0.5807
Epoch 8/50
1600/1600 [==============================] - 9s 6ms/step - loss: -0.5833 - dice_coef: 0.5833 - val_loss: -0.5826 - val_dice_coef: 0.5826
Epoch 9/50
1600/1600 [==============================] - 9s 6ms/step - loss: -0.5850 - dice_coef: 0.5850 - val_loss: -0.5809 - val_dice_coef: 0.5809
Epoch 10/50
 160/1600 [==>...........................] - ETA: 7s - loss: -0.6067 - dice_coef: 0.6067

1600/1600 [==============================] - 9s 6ms/step - loss: -0.5845 - dice_coef: 0.5845 - val_loss: -0.5828 - val_dice_coef: 0.5828
Epoch 11/50
1600/1600 [==============================] - 9s 6ms/step - loss: -0.5854 - dice_coef: 0.5854 - val_loss: -0.5834 - val_dice_coef: 0.5834
Epoch 12/50
1600/1600 [==============================] - 9s 6ms/step - loss: -0.5865 - dice_coef: 0.5865 - val_loss: -0.5840 - val_dice_coef: 0.5840
Epoch 13/50
1600/1600 [==============================] - 9s 6ms/step - loss: -0.5890 - dice_coef: 0.5890 - val_loss: -0.5851 - val_dice_coef: 0.5851
Epoch 14/50
1600/1600 [==============================] - 9s 6ms/step - loss: -0.5883 - dice_coef: 0.5883 - val_loss: -0.5875 - val_dice_coef: 0.5875
Epoch 15/50
 128/1600 [=>............................] - ETA: 7s - loss: -0.5741 - dice_coef: 0.5741

1600/1600 [==============================] - 9s 6ms/step - loss: -0.5914 - dice_coef: 0.5914 - val_loss: -0.5880 - val_dice_coef: 0.5880
Epoch 16/50
1600/1600 [==============================] - 9s 6ms/step - loss: -0.5937 - dice_coef: 0.5937 - val_loss: -0.5914 - val_dice_coef: 0.5914
Epoch 17/50
1600/1600 [==============================] - 9s 6ms/step - loss: -0.6016 - dice_coef: 0.6016 - val_loss: -0.5985 - val_dice_coef: 0.5985
Epoch 18/50
1600/1600 [==============================] - 9s 6ms/step - loss: -0.6182 - dice_coef: 0.6182 - val_loss: -0.6205 - val_dice_coef: 0.6205
Epoch 19/50
1600/1600 [==============================] - 9s 6ms/step - loss: -0.6327 - dice_coef: 0.6327 - val_loss: -0.6409 - val_dice_coef: 0.6409
Epoch 20/50
 128/1600 [=>............................] - ETA: 7s - loss: -0.6418 - dice_coef: 0.6418

1600/1600 [==============================] - 9s 6ms/step - loss: -0.6591 - dice_coef: 0.6591 - val_loss: -0.6610 - val_dice_coef: 0.6610
Epoch 21/50
1600/1600 [==============================] - 9s 6ms/step - loss: -0.6873 - dice_coef: 0.6873 - val_loss: -0.6923 - val_dice_coef: 0.6923
Epoch 22/50
1600/1600 [==============================] - 9s 6ms/step - loss: -0.7077 - dice_coef: 0.7077 - val_loss: -0.6993 - val_dice_coef: 0.6993
Epoch 23/50
1600/1600 [==============================] - 9s 6ms/step - loss: -0.7186 - dice_coef: 0.7186 - val_loss: -0.6856 - val_dice_coef: 0.6856
Epoch 24/50
1600/1600 [==============================] - 9s 6ms/step - loss: -0.7159 - dice_coef: 0.7159 - val_loss: -0.7172 - val_dice_coef: 0.7172
Epoch 25/50
 128/1600 [=>............................] - ETA: 7s - loss: -0.7539 - dice_coef: 0.7539

...

1600/1600 [==============================] - 9s 6ms/step - loss: -0.7671 - dice_coef: 0.7671 - val_loss: -0.7567 - val_dice_coef: 0.7567
Epoch 31/50
1600/1600 [==============================] - 9s 6ms/step - loss: -0.7723 - dice_coef: 0.7723 - val_loss: -0.7528 - val_dice_coef: 0.7528
Epoch 32/50
1600/1600 [==============================] - 9s 6ms/step - loss: -0.7731 - dice_coef: 0.7731 - val_loss: -0.7684 - val_dice_coef: 0.7684
Epoch 33/50
1600/1600 [==============================] - 9s 6ms/step - loss: -0.7840 - dice_coef: 0.7840 - val_loss: -0.7695 - val_dice_coef: 0.7695
Epoch 34/50
1600/1600 [==============================] - 9s 6ms/step - loss: -0.7870 - dice_coef: 0.7870 - val_loss: -0.7724 - val_dice_coef: 0.7724
Epoch 35/50
 128/1600 [=>............................] - ETA: 7s - loss: -0.7736 - dice_coef: 0.7736

1600/1600 [==============================] - 9s 6ms/step - loss: -0.7923 - dice_coef: 0.7923 - val_loss: -0.7765 - val_dice_coef: 0.7765
Epoch 36/50
1600/1600 [==============================] - 9s 6ms/step - loss: -0.7965 - dice_coef: 0.7965 - val_loss: -0.7866 - val_dice_coef: 0.7866
Epoch 37/50
1600/1600 [==============================] - 9s 6ms/step - loss: -0.7962 - dice_coef: 0.7962 - val_loss: -0.7901 - val_dice_coef: 0.7901
Epoch 38/50
1600/1600 [==============================] - 9s 6ms/step - loss: -0.8061 - dice_coef: 0.8061 - val_loss: -0.7830 - val_dice_coef: 0.7830
Epoch 39/50
1600/1600 [==============================] - 9s 6ms/step - loss: -0.8082 - dice_coef: 0.8082 - val_loss: -0.7965 - val_dice_coef: 0.7965
Epoch 40/50
 128/1600 [=>............................] - ETA: 7s - loss: -0.8327 - dice_coef: 0.8327

1600/1600 [==============================] - 9s 6ms/step - loss: -0.8115 - dice_coef: 0.8115 - val_loss: -0.7921 - val_dice_coef: 0.7921
Epoch 41/50
1600/1600 [==============================] - 9s 6ms/step - loss: -0.8122 - dice_coef: 0.8122 - val_loss: -0.7997 - val_dice_coef: 0.7997
Epoch 42/50
1600/1600 [==============================] - 9s 6ms/step - loss: -0.8143 - dice_coef: 0.8143 - val_loss: -0.8042 - val_dice_coef: 0.8042
Epoch 43/50
1600/1600 [==============================] - 9s 6ms/step - loss: -0.8124 - dice_coef: 0.8124 - val_loss: -0.8007 - val_dice_coef: 0.8007
Epoch 44/50
1600/1600 [==============================] - 9s 6ms/step - loss: -0.8191 - dice_coef: 0.8191 - val_loss: -0.8075 - val_dice_coef: 0.8075
Epoch 45/50
 128/1600 [=>............................] - ETA: 7s - loss: -0.8297 - dice_coef: 0.8297

1600/1600 [==============================] - 9s 6ms/step - loss: -0.8249 - dice_coef: 0.8249 - val_loss: -0.8038 - val_dice_coef: 0.8038
Epoch 46/50
1600/1600 [==============================] - 9s 6ms/step - loss: -0.8280 - dice_coef: 0.8280 - val_loss: -0.8139 - val_dice_coef: 0.8139
Epoch 47/50
1600/1600 [==============================] - 9s 6ms/step - loss: -0.8292 - dice_coef: 0.8292 - val_loss: -0.8162 - val_dice_coef: 0.8162
Epoch 48/50
1600/1600 [==============================] - 9s 6ms/step - loss: -0.8256 - dice_coef: 0.8256 - val_loss: -0.8128 - val_dice_coef: 0.8128
Epoch 49/50
1600/1600 [==============================] - 9s 6ms/step - loss: -0.8271 - dice_coef: 0.8271 - val_loss: -0.8155 - val_dice_coef: 0.8155
Epoch 50/50
 128/1600 [=>............................] - ETA: 7s - loss: -0.8341 - dice_coef: 0.8341

1600/1600 [==============================] - 9s 6ms/step - loss: -0.8332 - dice_coef: 0.8332 - val_loss: -0.8193 - val_dice_coef: 0.8193
Sorensen–Dice_coefficient: 0.8193368458747864


In [0]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Score:', score[1])

Score: 0.8193368458747864
